### Preprocessing Data

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import cv2
import random
import os
import pandas as pd
from PIL import Image
from skimage import exposure
from scipy.ndimage import gaussian_filter
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight

2024-04-13 05:32:46.476355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 05:32:46.634226: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 05:32:47.426350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-13 05:32:47.426540: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [3]:
# Load the dataset
data = pd.read_csv("../../aws_s3/DDR+APTOS_TRAIN_TEST/train_test_DDR_APTOS.csv")

filtered_data = data[data["Data_source"] == "DDR"]
# Split the dataset into training and testing sets
train_data = filtered_data[filtered_data["Split"] == "Train"]
test_data = filtered_data[filtered_data["Split"] == "Test"]

In [4]:
# Define output directories
luminosity_output_dir_train = "luminosity_images_train/"
cropped_output_dir_train = "cropped_images_train/"
normalized_output_dir_train = "normalized_images_train/"
luminosity_output_dir_test = "luminosity_images_test/"
cropped_output_dir_test = "cropped_images_test/"
normalized_output_dir_test = "normalized_images_test/"

# Create directories if they don't exist
os.makedirs(luminosity_output_dir_train, exist_ok=True)
os.makedirs(cropped_output_dir_train, exist_ok=True)
os.makedirs(normalized_output_dir_train, exist_ok=True)
os.makedirs(luminosity_output_dir_test, exist_ok=True)
os.makedirs(cropped_output_dir_test, exist_ok=True)
os.makedirs(normalized_output_dir_test, exist_ok=True)

In [5]:
def luminosity_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)

    # Merge back LAB channels
    lab_eq = cv2.merge((l_eq, a, b))
    enhanced_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    # Image noise removal using Gaussian filter
    filtered_image = cv2.GaussianBlur(enhanced_image, (5, 5), 0)

    return filtered_image


In [6]:
def crop_image(img, threshold=20, resize_flag=False, desired_size=(512, 512)):
    img_np = np.array(img)
    x_dim = img_np.shape[0]
    y_dim = img_np.shape[1]
    pixel_sums = img_np.sum(axis=2)
    x_arr = pixel_sums.sum(axis=1)
    y_arr = pixel_sums.sum(axis=0)
    x_start = np.where(x_arr > threshold * y_dim)[0][0]
    x_end = np.where(x_arr > threshold * y_dim)[0][-1]
    y_start = np.where(y_arr > threshold * x_dim)[0][0]
    y_end = np.where(y_arr > threshold * x_dim)[0][-1]
    new_img = img_np[x_start:x_end, y_start:y_end]
    new_img = Image.fromarray(new_img)
    if resize_flag:
        new_img = new_img.resize(desired_size)
    return new_img

In [7]:
def normalize_image(img):
    # Convert image to numpy array
    img_np = np.array(img)

    # Calculate mean and standard deviation (std) channel-wise
    mean_channels = np.mean(img_np, axis=(0, 1))
    std_channels = np.std(img_np, axis=(0, 1))

    # Normalize each channel separately
    normalized_image = np.zeros_like(img_np, dtype=np.float32)
    for channel in range(img_np.shape[2]):
        normalized_image[:, :, channel] = (
            img_np[:, :, channel] - mean_channels[channel]
        ) / std_channels[channel]

    # Scale values to be within [0, 255]
    normalized_image = (
        (normalized_image - np.min(normalized_image))
        / (np.max(normalized_image) - np.min(normalized_image))
        * 255
    )

    # Clip and return the normalized image
    normalized_image = np.clip(normalized_image, 0, 255)
    return normalized_image.astype(np.uint8)

In [8]:
X_train = []
y_train = []


# Function to get the correct image path with extension
def get_image_path(base_dir, img_name):
    for ext in ["jpg", "png"]:
        if "jpg" in img_name or "png" in img_name:
            temp_path = f"{base_dir}/{img_name}"
        else:
            temp_path = f"{base_dir}/{img_name}.{ext}"
        if os.path.exists(temp_path):
            return temp_path
    return None  # Return None if no file is found


# Load and preprocess each image
for index, row in train_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/train_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Change Luminosity and do noise removal for the image
    filtered_image = luminosity_image(image_path)

    # Crop and resize the image
    luminosity_img = Image.fromarray(filtered_image)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(512, 512)
    )

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)


    # Save cropped image
    # if "jpg" in img_name:
    #     cropped_img_name = f"{img_name}_{index+1}.jpg"
    # elif "png" in img_name:
    #     cropped_img_name = f"{img_name}_{index+1}.png"
    # cropped_img_name = f"{img_name}"
    # cropped_img_path = os.path.join(cropped_output_dir_train, cropped_img_name)
    # cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    # normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    # if "jpg" in img_name:
    #     normalized_img_name = f"norm_{img_name}_{index+1}.jpg"
    # elif "png" in img_name:
    #     normalized_img_name = f"norm_{img_name}_{index+1}.png"
    normalized_img_name = f"norm_{img_name}"
    normalized_img_path = os.path.join(normalized_output_dir_train, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))
    X_train.append(normalized_img)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

Corrupt JPEG data: 34 extraneous bytes before marker 0xd9
Corrupt JPEG data: 35 extraneous bytes before marker 0xd9
Corrupt JPEG data: 38 extraneous bytes before marker 0xd9


In [9]:
X_test = []
y_test = []

# Load and preprocess each image
for index, row in test_data.iterrows():
    img_name, label = row["Image_ID"], row["Retinopathy_Grade"]
    image_path = get_image_path(
        "../../aws_s3/DDR+APTOS_TRAIN_TEST/test_new", img_name
    )  # Dynamically get the correct image path

    if image_path is None:
        print(f"Image file for {img_name} not found in JPG or PNG format.")
        continue  # Skip this iteration if the file doesn't exist

    # Preprocess the image
    luminosity_img_path = luminosity_image(image_path, luminosity_output_dir_test)

    # Crop and resize the image
    luminosity_img = Image.open(luminosity_img_path)
    cropped_resized_img = crop_image(
        luminosity_img, resize_flag=True, desired_size=(512, 512)
    )

    # Save cropped image
    cropped_img_name = f"{img_name}"
    cropped_img_path = os.path.join(cropped_output_dir_test, cropped_img_name)
    cropped_resized_img.save(cropped_img_path)

    # Normalize the image
    normalized_img = normalize_image(cropped_resized_img)

    # Save normalized image
    normalized_img_name = f"norm_{img_name}"
    normalized_img_path = os.path.join(normalized_output_dir_test, normalized_img_name)
    cv2.imwrite(normalized_img_path, cv2.cvtColor(normalized_img, cv2.COLOR_RGB2BGR))

    # Append to X_test and y_test
    X_test.append(normalized_img)
    y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)

### CNN512 Model (no dropout)

In [10]:
# from tensorflow.keras.layers import (
#     Conv2D,
#     MaxPooling2D,
#     Flatten,
#     Dense,
#     BatchNormalization,
#     ZeroPadding2D,
#     Activation,
#     Dropout,
# )
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import SGD
# from sklearn.utils.class_weight import compute_class_weight
# from tensorflow.keras.callbacks import LearningRateScheduler
# import math

# # Define the CNN512 model architecture
# model = Sequential()

# # Input Layer (Zero Padding)
# model.add(ZeroPadding2D(padding=(2, 2), input_shape=(512, 512, 3)))

# # Layer 1, 2, 3
# model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 4
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 5, 6, 7
# model.add(Conv2D(64, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 8
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 9, 10, 11
# model.add(Conv2D(96, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 12
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 13, 14, 15
# model.add(Conv2D(96, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 16
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 17, 18, 19
# model.add(Conv2D(128, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 20
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 21, 22, 23
# model.add(Conv2D(200, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 24
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 25
# model.add(Flatten())

# # Layer 26 (Dropout)
# # model.add(Dropout(0.5))

# # Layer 27, 28, 29
# model.add(Dense(1000))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 30 (Dropout)
# # model.add(Dropout(0.5))

# # Layer 31, 32, 33
# model.add(Dense(500))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 34 (Dropout)
# # model.add(Dropout(0.5))

# # Layer 35
# model.add(Dense(5, activation="softmax"))  # Assuming 4 classes for Retinopathy grade

### CNN512 Model (with dropout after FC layers)

In [11]:
# from tensorflow.keras.layers import (
#     Conv2D,
#     MaxPooling2D,
#     Flatten,
#     Dense,
#     BatchNormalization,
#     ZeroPadding2D,
#     Activation,
#     Dropout,
# )
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import SGD
# from sklearn.utils.class_weight import compute_class_weight
# from tensorflow.keras.callbacks import LearningRateScheduler
# import math

# # Define the CNN512 model architecture
# model = Sequential()

# # Input Layer (Zero Padding)
# model.add(ZeroPadding2D(padding=(2, 2), input_shape=(512, 512, 3)))

# # Layer 1, 2, 3
# model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 4
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 5, 6, 7
# model.add(Conv2D(64, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 8
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 9, 10, 11
# model.add(Conv2D(96, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 12
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 13, 14, 15
# model.add(Conv2D(96, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 16
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 17, 18, 19
# model.add(Conv2D(128, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 20
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 21, 22, 23
# model.add(Conv2D(200, (3, 3)))
# model.add(BatchNormalization())
# model.add(Activation("relu"))

# # Layer 24
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # Layer 25
# model.add(Flatten())

# # Layer 27, 28, 29
# model.add(Dense(1000))
# model.add(BatchNormalization())
# model.add(Activation("relu"))
# model.add(Dropout(0.5))

# # Layer 31, 32, 33
# model.add(Dense(500))
# model.add(BatchNormalization())
# model.add(Activation("relu"))
# model.add(Dropout(0.5))

# # Layer 35
# model.add(Dense(5, activation="softmax"))  # Assuming 4 classes for Retinopathy grade

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    BatchNormalization,
    ZeroPadding2D,
    Activation,
    Dropout,
)
from tensorflow.keras.optimizers import SGD

# Create the Sequential model
model512 = Sequential()

# Input Layer (Zero Padding)
model512.add(ZeroPadding2D(padding=(2, 2), input_shape=(512, 512, 3)))

# Layer 1
model512.add(Conv2D(32, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 2
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3
model512.add(Conv2D(64, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 4
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 5
model512.add(Conv2D(96, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 6
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 7
model512.add(Conv2D(96, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 8
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 9
model512.add(Conv2D(128, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 10
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 11
model512.add(Conv2D(200, (3, 3)))
model512.add(BatchNormalization())
model512.add(Activation("relu"))

# Layer 12
model512.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 13 (Dropout)
model512.add(Dropout(0.5))

# Layer 14
model512.add(Dense(1000))
model512.add(BatchNormalization())
model512.add(Activation("relu"))
model512.add(Dropout(0.5))

# Layer 15
model512.add(Dense(500))
model512.add(BatchNormalization())
model512.add(Activation("relu"))
model512.add(Dropout(0.5))

# Layer 16 (Output Layer)
model512.add(Dense(5, activation="softmax"))  # Assuming 5 classes for classification

# Define the optimizer with specified learning rate and momentum
learning_rate = 1e-1
optimizer = SGD(learning_rate=learning_rate, momentum=0.9)

# Compile the model with SGD optimizer and specified loss function
model512.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)


# Define the learning rate schedule
def lr_schedule(epoch):
    return learning_rate * math.pow(0.5, math.floor((1 + epoch) / 10))


# Define the learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Fit the model to the training data
history = model512.fit(
    X_train,
    y_train,
    epochs=10,  # Assuming 10 epochs for training
    batch_size=16,  # Assuming batch size of 32
    callbacks=[lr_scheduler],  # Learning rate scheduler callback
    class_weight="auto",  # Auto compute class weights
    verbose=1,
)  # Show training progress

# Evaluate the model on the test data
test_loss, test_accuracy = model512.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

2024-04-13 06:13:58.316826: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7879262208 exceeds 10% of free system memory.
2024-04-13 06:14:12.329332: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.34GiB (rounded to 7879262208)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-13 06:14:12.329381: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2024-04-13 06:14:12.329399: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 63, Chunks in use: 63. 15.8KiB allocated for chunks. 15.8KiB in use in bin. 5.8KiB client-requested in use in bin.
2024-04-13 06:14:12.329410: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# from tensorflow.keras.optimizers import SGD

# # base learning rate
# base_learning_rate = 1e-4
# # maximum learning rate
# max_learning_rate = 1e-2

# # Create an instance of SGD optimizer with initial learning rate
# optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9, clipnorm=1.0)

# # create class weight
# classes = np.unique(y_train)
# class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
# class_weight_dict = dict(zip(classes, class_weights))


# # create triangular schedule
# def triangular_schedule(epoch):
#     """Triangular learning rate scheduler."""
#     cycle_length = 10  # Define the length of a cycle
#     cycle = math.floor(1 + epoch / (2 * cycle_length))
#     x = abs(epoch / cycle_length - 2 * cycle + 1)
#     lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
#     return lr


# # When fitting the model, include the learning rate scheduler callback
# lr_scheduler = LearningRateScheduler(triangular_schedule)

In [ ]:
# model.compile(
#     optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )

# print("Fitting the top layer of the model")
# model.fit(
#     X_train,
#     y_train,
#     epochs=10,
#     batch_size=32,
#     class_weight=class_weight_dict,
#     callbacks=[lr_scheduler],
# )

Fitting the top layer of the model


2024-04-13 01:26:42.931320: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7879262208 exceeds 10% of free system memory.
2024-04-13 01:26:47.177854: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7879262208 exceeds 10% of free system memory.


Epoch 1/10


2024-04-13 01:26:52.265105: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8902
2024-04-13 01:26:52.381890: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-13 01:26:53.061359: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-13 01:26:53.061428: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


314/314 [==============================] - 39s 115ms/step - loss: 1.8181 - accuracy: 0.2720 - lr: 1.0000e-04
Epoch 2/10
314/314 [==============================] - 36s 114ms/step - loss: 1.5985 - accuracy: 0.3644 - lr: 0.0011
Epoch 3/10
314/314 [==============================] - 36s 114ms/step - loss: 1.4761 - accuracy: 0.4182 - lr: 0.0021
Epoch 4/10
314/314 [==============================] - 36s 114ms/step - loss: 1.3889 - accuracy: 0.4297 - lr: 0.0031
Epoch 5/10
314/314 [==============================] - 36s 114ms/step - loss: 1.3507 - accuracy: 0.4772 - lr: 0.0041
Epoch 6/10
314/314 [==============================] - 36s 114ms/step - loss: 1.2664 - accuracy: 0.4996 - lr: 0.0050
Epoch 7/10
314/314 [==============================] - 36s 114ms/step - loss: 1.1420 - accuracy: 0.5217 - lr: 0.0060
Epoch 8/10
314/314 [==============================] - 36s 114ms/step - loss: 1.0190 - accuracy: 0.5637 - lr: 0.0070
Epoch 9/10
314/314 [==============================] - 36s 114ms/step - loss: 0.

In [ ]:
# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print("Test Accuracy:", test_accuracy)

79/79 [==============================] - 3s 34ms/step - loss: 1.3069 - accuracy: 0.4674
Test Accuracy: 0.46743908524513245
